# SSCN and PointNet Layer Implementation Assignment

SSCN uses the preprocessing method used in [Volumetric and Multi-View CNNs for Object Classification on 3D Data](https://arxiv.org/abs/1604.03265)

We are going to use a simpler method that doesn't require another CNN, where we voxelize the given .off files and project them from multiple angles to get a 2d image
we can use.

### Download binvox and dataset

In [2]:
!test -f binvox || wget https://www.patrickmin.com/binvox/linux64/binvox

In [3]:
!chmod +x binvox

In [5]:
!./download_dataset.sh > /dev/null

### Runs script to voxelize `.off` file
`/dev/null` pipe is to stop it from breaking stdout, if the program is not working, remove it to debug

In [4]:
!./voxelize.sh ModelNet10/bed/train/bed_0010.off > /dev/null

In [1]:
!g++ -std=c++11 read_vox.cpp -o read_vox

In [3]:
!./read_vox ModelNet10/bed/train/bed_0010.binvox

reading binvox version 1
  read 143417 voxels
Writing voxel data to ASCII file...
done



# Parse voxel file into C++ memory
This is for SSCN since we need 2d Projected 3d objects

In [1]:
#include<fstream>
#include<iostream>
#include<vector>
#include<string>
using namespace std;



In [2]:
typedef vector<vector<vector<char>>> v3d;
typedef vector<vector<char>> v2d;
typedef vector<vector<float>> v2df;


Read all voxel data from voxels.txt and output in 3d vector

In [3]:
v3d read_vox(){
    ifstream voxels("voxels.txt");
    string t;
    int x, y, z;
    // ignore irrelevant 
    voxels >> t >> t >> t >> t;
    voxels >> x >> y >> z;
    voxels >> t >> t >> t >> t;
    voxels >> t >> t >> t;
    vector<vector<vector<char>>> data(x, vector<vector<char>>(z, vector<char>(y, 0)));
    // Read data into vec
    // format is [x][z][y]
    for(int i = 0; i < x; i++){
        for(int j = 0; j < z; j++){
            for(int k = 0; k < y; k++){
                char v;
                voxels >> v;
                data[i][j][k] = v;
            }
        }
        
    }
    return data;
}


In [4]:
char max(char x, char y) {
    return x > y ? x : y;
}

In [5]:
// Project onto x y or z
// sweeps across given axis
// 0 = x, 1 = z, 2 = y
v2d project(const v3d &data, int axis, int x_size, int y_size){
    v2d out(x_size, vector<char>(y_size, 0));
    if(axis == 0){
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[i][j][k]);
                }
            }
        }
    } else if(axis == 1){
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[j][i][k]);
                }
            }
        }
    } else{
        for(int i = 0; i < data.size(); i++){
            for(int j = 0; j < data[0].size(); j++){
                for(int k = 0; k < data[0][0].size(); k++){
                    out[j][k] = max(out[j][k], data[j][k][i]);
                }
            }
        }
    }
    return out;
}

In [6]:
void print_2d(const v2d &data){
    for(auto i : data){
        for (auto j : i){
            cout << j << " ";
        }
        cout << endl;
    }
}


In [7]:

void print_2df(const v2df &data){
    for(auto i : data){
        for (auto j : i){
            cout << j << "\t";
        }
        cout << endl;
    }
}

In [8]:
// Get our 3d vector of intputs
v3d data = read_vox();
int x = data.size();
int z = data[0].size();
int y = data[0][0].size();
cout << "x: " << x << endl;
cout << "y: " << y << endl;
cout << "z: " << z << endl;
v2d p = project(data, 2, x, y);


x: 256
y: 256
z: 256


## SCNN Assignment
The goal of this assignment will be to write a 2D sparse convolution. Sparse convolutions are essentially the same as normal convolutions, but values in the center of a kernel will remain zero to prevent exploding of sparse data. 

For simplicity, your kernel will be 5x5, and it will be using

In [9]:
// Convert byte array to floats
typedef vector<vector<float>> v2df;
v2df pf;
for(int i = 0; i < x; i ++){
    vector<float> v;
    for(int j = 0; j < y; j++){
        v.push_back((float) p[i][j]);
    }
    pf.push_back(v);
}

In [10]:
//ACTION: Complete this sparse convolution implementation
v2df sparse_convolution(const v2df& data, const v2df& kernel){
    // Dimensions of input data
    int dataHeight = data.size();
    int dataWidth = data[0].size();
    
    // Dimensions of kernel (fixed to 5x5)
    const int kernelSize = 5;
    const int pad = kernelSize / 2;

    // Output dimensions
    int outputHeight = dataHeight;
    int outputWidth = dataWidth;

    // Initialize output with zeros
    v2df output(outputHeight, vector<float>(outputWidth, 0.0f));

    // Perform 2D convolution
    // REMOVE THIS FOR STUDENT TO FILL IN FOR ASSIGNMENT
    for (int i = 0; i < dataHeight; ++i) {
        for (int j = 0; j < dataWidth; ++j) {
            // Accumulate convolution sum
            float sum = 0.0f;
            if (data[i][j] != 0){
                for (int ki = 0; ki < kernelSize; ++ki) {
                    for (int kj = 0; kj < kernelSize; ++kj) {
                        // Determine input indices
                        int ni = i + ki - pad;
                        int nj = j + kj - pad;

                        // Check for boundary conditions
                        if (ni >= 0 && ni < dataHeight && nj >= 0 && nj < dataWidth) {
                            sum += data[ni][nj] * kernel[ki][kj];
                        }
                    }
                }
            // Store result in output
                output[i][j] = sum;
            }
        }
    }

    return output;
}

In [11]:
v2df kernel;
for(int i = 0; i < 5; i++){
    vector<float> v;
    for(int j = 0; j < 5; j++){
        v.push_back(i + j);
    }
    kernel.push_back(v);
}
// Running on layer
v2df result = sparse_convolution(pf, kernel);




### Line example
If your convolution is working properly, this line example should still produce2 intersection lines after convolution instead of exploding.

In [13]:
v2df cross;
for(int i = 0; i < 20; i++){
    vector<float> v;
    for(int j = 0; j < 20; j++){
        if (i == 5){
            v.push_back(0.24);

        } 
        else if(j == 3){
            v.push_back(0.12);
        }
        else{
            v.push_back(0);
        }
    }
    cross.push_back(v);
}

v2df k;
for(int i = 0; i < 5; i++){
    vector<float> v;
    for(int j = 0; j < 5; j++){
        v.push_back(1);
    }
    k.push_back(v);
}
v2df cross_res = sparse_convolution(cross, k);
print_2df(cross_res);

// Verification number
float sum = 0;
for(int i = 0; i < 20; i++){
    sum += cross_res[5][i];
}
cout << "row sum: " << sum << endl;
sum = 0;
for(int i = 0; i < 20; i++){
    sum += cross_res[i][3];
}
cout << "col sum: " << sum << endl;

0	0	0	0.36	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.48	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	1.68	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	1.68	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0.72	1.44	1.68	1.68	1.68	1.68	1.2	1.2	1.2	1.2	1.2	1.2	1.2	1.2	1.2	1.2	1.2	1.2	0.96	0.72	
0	0	0	1.68	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	1.68	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.6	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.48	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
0	0	0	0.36	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
row sum: 24.96
col sum: 16.68


### Expected Results
row sum: 24.96

col sum: 16.68

In the output grid, all value not along row 5 or column 3 should be zero.

# PointNet++ Assignment

### Helper Functions
These are functions that are implemented for you.

In [13]:
#include <iostream>
#include <fstream>
#include <vector>
#include <cmath>
#include <limits>
#include <algorithm>
#include <random>
using namespace std;

In [14]:
// Generate randome numbers for weights
float random_float(float min, float max) {
    int seed = 1;
    mt19937 gen(seed);
    uniform_real_distribution<> dis(min, max);
    return dis(gen);
}

In [15]:
// Activation function: ReLU
float relu(float x) {
    return max(0.0f, x);
}

In [16]:
float dot_product(const vector<float>& p1, const vector<float>& p2) {
    return p1[0] * p2[0] + p1[1] * p2[1] + p1[2] * p2[2];
}

In [17]:
// Matrix-vector multiplication
auto mat_vec_mul(const vector<vector<float>>& mat, const vector<float>& vec) {
    vector<float> result(mat.size(), 0.0f);
    for (size_t i = 0; i < mat.size(); ++i) {
        for (size_t j = 0; j < mat[0].size(); ++j) {
            result[i] += mat[i][j] * vec[j];
        }
    }
    return result;
}

In [18]:
// Add bias vector to output
auto add_bias(const vector<float>& vec, const vector<float>& bias) {
    vector<float> result = vec;
    for (size_t i = 0; i < vec.size(); ++i) {
        result[i] += bias[i];
    }
    return result;
}

### Point Distance
Fill in the function `euclidean_distance` to find the distance between two 3D points
HINT:
$$
p_1 = (x_1, y_1, z_1)
$$
$$
p_2 = (x_2, y_2, z_2)
$$
$$
d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2}
$$


In [19]:
// ACTION: Write a function to find the distance between two points
float euclidean_distance(const vector<float>& p1, const vector<float>& p2) {
    // DELETE THIS PART IN FINAL ASSIGNMENT
    return sqrt(pow(p1[0] - p2[0], 2) + pow(p1[1] - p2[1], 2) + pow(p1[2] - p2[2], 2));
}

### Farthest Point Sampling
This is a sampling technique where given points $\{ x_1, x_2, \dots, x_n\}$, we need to find a subset of these points $\{x_{i_1},x_{i_2}, \dots, x_{i_m}\}$ where $x_{i_j}$ is the furthest point from the set $\{x_{i_1},x_{i_2}, \dots, x_{j-1}\}$. We use this technique to ensure an even coverage points across the whole pointcloud.

In [20]:
// ACTION: Implement farthest point sampling
// returns group of vectors, vector<float>
auto farthest_point_sampling(const vector<vector<float>>& points, int num_samples) {
    vector<int> sampled_indices;
    int n = points.size();
    vector<float> distances(n, numeric_limits<float>::infinity());

    // Randomly select the first point
    sampled_indices.push_back(rand() % n);

    // DELETE THIS PART IN FINAL ASSIGNMENT
    for (int i = 1; i < num_samples; ++i) {
        float max_dist = -1;
        int farthest_point = -1;

        // Compute distances from the sampled points
        for (int j = 0; j < n; ++j) {
            float min_dist = numeric_limits<float>::infinity();
            for (int k : sampled_indices) {
                float dist = euclidean_distance(points[j], points[k]);
                min_dist = min(min_dist, dist);
            }
            distances[j] = min_dist;
            if (distances[j] > max_dist) {
                max_dist = distances[j];
                farthest_point = j;
            }
        }
        sampled_indices.push_back(farthest_point);
    }
    return sampled_indices;
}

### Group Neighbors
Group points into neighborhoods based on radius. We input points, centroids and a radius. We group all of the points around the centroids, and those groups around the centroids get added to our grouped_points variable.

In [21]:
// ACTION: Fill in the function to group points around the centroids within the radius.
auto group_neighbors(const vector<vector<float>>& points, const vector<vector<float>>& centroids, float radius) {
    vector<vector<vector<float>>> grouped_points;
    
    // DELETE THIS PART IN FINAL ASSIGNMENT
    for (const auto& centroid : centroids) {
        vector<vector<float>> neighbors;
        for (const auto& point : points) {
            if (euclidean_distance(centroid, point) <= radius) {
                neighbors.push_back(point);
            }
        }
        grouped_points.push_back(neighbors);
    }
    return grouped_points;
}

### Multi-layer Perceptron (MLP)
Now we need to put all of these functions together. We will be randomly generating weights, since we won't be implementing backpropagation to update weights in this assignment. 
Our MLP has the following steps
1. Aggregate point features with average pooling
2. Perform matrix vector multiplication on aggregated point features and weights
3. Add biases

In [22]:
// MLP Function
// Note: we do not learn our weights and biases here and just use random values
// ACTION: fill in the action blocks of this function
auto mlp(const vector<vector<float>>& points, int input_dim, int output_dim) {
    // Number of points in the group
    size_t num_points = points.size();

    // Random number generator for weights and biases
    int seed = 1;
    mt19937 gen(seed);
    uniform_real_distribution<> dis(-0.1, 0.1);

    // Initialize weights and biases for a single-layer MLP
    vector<vector<float>> weights(output_dim, vector<float>(input_dim, 0.0f));
    vector<float> biases(output_dim, 0.0f);

    // Fill weights and biases with random values
    for (size_t i = 0; i < output_dim; ++i) {
        for (size_t j = 0; j < input_dim; ++j) {
            weights[i][j] = dis(gen);
        }
        biases[i] = dis(gen);
    }

    // Aggregate point features (e.g., average pooling)
    // This is the sum portion of the average pooling
    vector<float> aggregated_features(input_dim, 0.0f);
    for (const auto& point : points) {
        // ACTION: Fill in the interior of this loop
        for (int i = 0; i < input_dim; ++i) {
            // DELETE THIS PART FOR ASSIGNMENT RELEASE
            aggregated_features[i] += point[i];
        }
    }
    
    // Normalize by the number of points
    // This is the division portion of the average pooling
    // ACTION: Fill in the interior of this loop
    for (int i = 0; i < input_dim; ++i) {
        
        // DELETE THIS PART FOR ASSIGNMENT RELEASE
        aggregated_features[i] /= num_points; 
    }

    // Apply MLP: matrix-vector multiplication + bias addition + activation
    // ACTION: Perform matrix vector multiplication
    vector<float> output = mat_vec_mul(weights, aggregated_features); // REMOVE RIGHT SIDE OF EQUALS FOR RELEASE
    output = add_bias(output, biases); // keep this part

    // Apply ReLU activation
    for (auto& val : output) {
        val = relu(val);
    }

    return output; // Return the processed feature vector
}

### Max Pooling
Perform max pooling on a list of features. This will be the maximum of all the dimensions, so max pooling of $\{(1, 2, 3), (3, 2, 1)\} = (3, 2, 3)$

In [23]:
// Max pooling operation
// ACTION: Implement Max pooling
auto max_pool(const vector<vector<float>>& features) {
    vector<float> pooled(features[0].size(), -numeric_limits<float>::infinity());
    // DELETE THIS PART FOR ASSIGNMENT RELEASE
    for (const auto& feature : features) {
        for (size_t i = 0; i < feature.size(); ++i) {
            pooled[i] = max(pooled[i], feature[i]);
        }
    }
    return pooled;
}

## PointNet++ Forward Pass
This portion will implement forward pass using the functions you implemented earlier. Here's the steps:
1. Obtain sample points (using FPS)
2. Use the indexes of these sampled points to obtain the centroids
3. Group the points around the centroids (sampled_points) with group_neighbors
4. Process each region with MLP
5. Perform max pooling over the features from MLP
6. Update features for next layer

In [24]:
// PointNet++ Core Algorithm
auto pointnet_plus_plus(const vector<vector<float>>& input_points, int num_layers, int k_neighbors, float radius) {
    vector<vector<float>> current_points = input_points;
    vector<float> final_features;

    // Iterate through the layers
    for (int layer = 0; layer < num_layers; ++layer) {
        // Sample centroids using Farthest Point Sampling (FPS)
        vector<int> sampled_indices = farthest_point_sampling(current_points, k_neighbors);

        // Extract centroids
        vector<vector<float>> sampled_points;
        for (int idx : sampled_indices) {
            sampled_points.push_back(current_points[idx]);
        }

        // Group neighboring points for each centroid
        vector<vector<vector<float>>> grouped_points = group_neighbors(current_points, sampled_points, radius);

        // Process each local region using an MLP (PointNet subroutine)
        vector<vector<float>> local_features;
        for (const auto& group : grouped_points) {
            local_features.push_back(mlp(group, 3, 128)); // Input 3D points, output feature size 128
        }

        // Max pooling over local features
        vector<float> aggregated_features = max_pool(local_features);

        // Update current points with aggregated features for the next layer
        current_points.clear();
        current_points.push_back(aggregated_features);  // Simplified: use aggregated feature as the next layer's input
    }

    // Final feature aggregation via max pooling
    final_features = max_pool(current_points);

    return final_features;
}

## PointNet++ Testing
Next we will test your implementation of the functions. We will load in points from bed_0001 in ModelNet10.


In [25]:
auto load_points (){
    ifstream pstream("ModelNet10/bed/train/bed_0001.off");
    string off;
    int num_points, surface, mesh;
    vector<vector<float>> points;
    pstream >> off >> num_points >> surface >> mesh;
    for(int i = 0; i < num_points; i++){
        vector<float> p;
        for(int j = 0; j < 3; j++){
            float dim;
            pstream >> dim;
            p.push_back(dim);
        }
        points.push_back(p);
    }
    return points;
}

In [26]:
vector<vector<float>> points = load_points();
vector<float> final_features = pointnet_plus_plus(points, 5, 100, 100);
float sum = 0;
for(auto feature : final_features){
    sum += feature;
}
cout << "\nCHECK_SUM: " << sum;


CHECK_SUM: 3.05936

The expected value for CHECK_SUM on first pass is 3.05936